In [ ]:
# 구글 드라이브와 연결
# from google.colab import auth
# auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# 경로 변경
#!cd /content/gdrive/My\ Drive/pytest/
%cd /content/gdrive/My\ Drive/pytest/
# %cd "/content/gdrive/My Drive/Colab Notebooks/pytest"
#!pip install -U -q PyDrive

In [ ]:
!ls   # 파일의 리스트 보여주기

In [ ]:
# 데이터 확인
import pandas as pd
import os

data = pd.read_csv("income.csv")

print(data.columns.values)             # 모든 컬럼의 내용을 본다
print(data.head())                     # 컬럼의 구성 요소를 본다

In [ ]:
# 독립변수 분리
X = data.loc[:, 'age':'nativeCountry']
X.head()

In [ ]:
# age, educationNumber, hourPerWeek 그리고 income 제외하고 원-핫 인코딩
X = pd.get_dummies(X, columns=['workclass', 'education', 'maritalStatus', 
                               'occupation', 'relationship', 'race', 'gender', 'nativeCountry'])
print(X.columns.values)
X.head()

In [ ]:
# 종속변수 분리
y = data.loc[:, 'income']
y.head()

In [ ]:
# 데이터 확인
print('X.shape:', X.shape)    # 105 컬럼이 나와야 정상이다 
print('y.shape:', y.shape)

In [ ]:
# 훈련데이터와 테스트데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
# 모델 정의하기
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()                                              # 모델을 새로 정의

model.add(Dense(64, activation='relu', input_shape=(105,)))	      # 첫 번째 은닉층
model.add(Dense(32, activation='relu'))                           # 두 번째 은닉층
model.add(Dense(1, activation='sigmoid'))                 		    # 출력층

In [ ]:
# 모델 요약 출력
model.summary()

In [ ]:
# Compile & Train Model
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

# 모델 훈련
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Accuracy & Loss 확인
# history 딕셔너리 안에 있는 정확도와 손실값을 가져와 본다

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

print('Accuracy of each epoch:', acc)   # 훈련데이터 정확도
epochs = range(1, len(acc) +1)			    # range(1, 11)

In [ ]:
# Plotting Accuracy
import matplotlib.pyplot as plt

# 훈련데이터의 정확도에 비해 검증데이터의 정확도는 늦게 올라간다
plt.plot(epochs, acc, 'bo', label='Training Acc')
plt.plot(epochs, val_acc, 'b', label='Validation Acc')
plt.title('Training and validation accuracy')
plt.legend()

In [ ]:
# Plotting Loss
plt.figure()    # 새로운 그림을 그린다

# 훈련데이터의 손실값은 낮아지나, 검증데이터의 손실값은 높아진다
# 손실값은 오류값을 말한다. 예측과 정답의 차이를 거리 계산으로 구한 값이다
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
data = pd.read_csv("income.csv")
data_X = data.loc[:, 'age':'nativeCountry']         # 훈련데이터에서 사용된 독립변수
print('data_X.shape:', data_X.shape)                # (32561, 11) 32561 행이 있다

newdata = pd.read_csv("income_test.csv")            # 새로운 데이터를 부름
newdata_X = newdata.loc[:, 'age':'nativeCountry']   # 새로운 데이터의 독립변수
print('newdata_X.shape:', newdata_X.shape)          # (10, 11) 10 행이 있다

data_appended = newdata_X.append(data_X)            # 새로운 데이터에 훈련데이터를 연결한다
print('data_appended.shape:', data_appended.shape)  # (32571, 11) 두 데이터가 결합된 32571 행

# 결합된 데이터에 대하여 원-핫-인코딩
data_appended = pd.get_dummies(data_appended, columns=['workclass', 'education', 'maritalStatus', 
                               'occupation', 'relationship', 'race', 'gender', 'nativeCountry'])
print('data_appended.shape:', data_appended.shape)  # (32571, 105). 원-핫 인코딩 이후의 컬럼수가 모델에서 사용한 컬럼의 수 105개와 같음
newdata_X = data_appended.iloc[0:10,:]              # 새로운 데이터 부분 10행만 다시 잘라낸다 

In [ ]:
prediction = model.predict(newdata_X)
print("Result:", prediction)					       # 1이 될 확률을 보여준다